# Projeto de Classificação com redução da dimensionalidade com Giny

## 1. Importação de bibliotecas

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score

from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

RANDOM_STATE = 42

## 2. Leitura e visualização inicial dos dados

In [2]:
# Substitua pelo caminho real do seu dataset, se necessário
df = pd.read_csv('smart-contract-dataset.csv')
df.head()

,Unnamed: 0,hash_id,label,bytecode_len,Weight bytecode_character_6,Weight bytecode_character_0,Weight bytecode_character_8,Weight bytecode_character_4,Weight bytecode_character_5,Weight bytecode_character_2,...,bytecode_character_k,bytecode_character_P,Weight bytecode_character_g,bytecode_character_g,Weight bytecode_character_I,Weight bytecode_character_m,bytecode_character_I,bytecode_character_m,Weight bytecode_character_x,bytecode_character_x
0,24345,3cef4261255f49f8ee35cc104becee8de0e98ecb979c64...,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14128,76187d55675bfdc44d3a50de7e1bb425ca46f6141776d0...,0,240,0.079167,0.345833,0.070833,0.033333,0.058333,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12942,034cceceee0f4af4b83789d99fd2eca3c9b9d95cce0515...,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7017,9d4a05454bba830a8e46f1afab8ad74a9c17aec7f6f684...,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,21079,a505708c309efee083dd5f89dcac0f33273474f02330d4...,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df.describe()

,Unnamed: 0,label,bytecode_len,Weight bytecode_character_6,Weight bytecode_character_0,Weight bytecode_character_8,Weight bytecode_character_4,Weight bytecode_character_5,Weight bytecode_character_2,Weight bytecode_character_1,...,bytecode_character_k,bytecode_character_P,Weight bytecode_character_g,bytecode_character_g,Weight bytecode_character_I,Weight bytecode_character_m,bytecode_character_I,bytecode_character_m,Weight bytecode_character_x,bytecode_character_x
count,36671.000000,36671.000000,36671.000000,36671.000000,36671.000000,36671.000000,36671.000000,36671.000000,36671.000000,36671.000000,...,36671.000000,36671.000000,3.667100e+04,36671.000000,3.667100e+04,3.667100e+04,36671.000000,36671.000000,3.667100e+04,36671.000000
mean,18335.000000,0.266041,4494.265769,0.054926,0.172130,0.037272,0.018298,0.054504,0.032398,0.059910,...,0.000491,0.000027,4.070075e-08,0.000055,5.270022e-08,5.270022e-08,0.000545,0.000545,3.051743e-08,0.000382
std,10586.150197,0.441892,6552.745838,0.034178,0.116114,0.024645,0.013500,0.035380,0.022089,0.039021,...,0.054267,0.005222,7.794061e-06,0.010444,4.514558e-06,4.514558e-06,0.046705,0.046705,4.132268e-06,0.051695
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
25%,9167.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
50%,18335.000000,0.000000,1660.000000,0.066755,0.187323,0.041914,0.020610,0.061487,0.040076,0.070336,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
75%,27502.500000,1.000000,8104.000000,0.077086,0.251205,0.054111,0.025904,0.079975,0.045181,0.090361,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
max,36670.000000,1.000000,90356.000000,0.185859,0.693885,0.115854,0.138139,0.147041,0.192966,0.196352,...,7.000000,1.000000,1.492537e-03,2.000000,3.989627e-04,3.989627e-04,4.000000,4.000000,5.595524e-04,7.000000


## 3. Pré-processamento

In [4]:
# Separação entre features e target
X = df.iloc[:, 3:].select_dtypes(include="number")
y = df.label

# Split entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=RANDOM_STATE)

# Escalonamento dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 4. Redução de Dimensionalidade com Giny

In [5]:
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X, y)
importances = tree.feature_importances_
indices = np.argsort(importances)[::-1][:15]
selected_features = X.columns[indices]
print("Features selecionadas:", selected_features.tolist())

# Seleção das 15 features mais importantes
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

Features selecionadas: ['emit', 'ast_id', 'Weight bytecode_character_5', 'internal', 'require', 'from', 'will', 'string', 'bytecode_entropy', 'external', 'length', 'Opcode weight LT', 'ast_len_nodes', 'mapping', 'can']


## 7. Otimização de Hiperparâmetros com GridSearchCV (poly)

In [ ]:
param_grid = [
    #{'kernel': ['rbf'], 'C': [1, 10], 'gamma': ['scale', 0.01]},
    {'kernel': ['poly'], 'C': [1,10,100], 'gamma': ['scale'], 'degree': [2]},
    #{'kernel': ['sigmoid'], 'C': [1], 'gamma': ['scale']}
]

grid = GridSearchCV(SVC(random_state=RANDOM_STATE), param_grid, cv=3, scoring='precision', verbose=3)
grid.fit(X_train_selected, y_train)

print(f'Melhores parâmetros: {grid.best_params_}')
best_model = grid.best_estimator_

y_pred_grid = best_model.predict(X_test_selected)

print(classification_report(y_test, y_pred_grid))

Fitting 3 folds for each of 3 candidates, totalling 9 fits


## 8. Conclusão